<a href="https://www.kaggle.com/code/kathmer/telco-churn-rates-eda-business-analysis?scriptVersionId=296797034" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd

df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


The main question : Which customer segments are most at risk, and where is revenue concentration?

TASK 1 — Load, Inspect, Define

In [2]:
df.shape

(7043, 21)

In [3]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [5]:
df['SeniorCitizen'].unique()

array([0, 1])

TASK 2 — Metric Safety Prep

Create:

is_churned (boolean from Churn)

has_positive_charges (TotalCharges > 0)

In [6]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['SeniorCitizen'] = df['SeniorCitizen'].astype('boolean')

In [7]:
df['is_churned'] = df['Churn'] == 'Yes'
df['has_positive_charges'] = df['TotalCharges'] > 0

TASK 3 — Churn Risk Segmentation

Business question: Which customer segments churn at higher rates?


Checking by contract

In [8]:
contract_summary = df.groupby('Contract')['is_churned'].mean()

contract_summary

Contract
Month-to-month    0.427097
One year          0.112695
Two year          0.028319
Name: is_churned, dtype: float64

Month-to-month (~42.7%) → highest churn (low commitment, easy exit)

One-year (~11.3%)

Two-year (~2.8%) → lowest churn (high commitment)

That pattern is a strong signal that: Contract length is a major churn driver, not noise.

Checking by internet service

In [9]:
internet_summary = df.groupby('InternetService')['is_churned'].mean().sort_values(ascending=False)
internet_summary

InternetService
Fiber optic    0.418928
DSL            0.189591
No             0.074050
Name: is_churned, dtype: float64

Interpretation:

Fiber optic (~41.9%) → highest churn

DSL (~19.0%)

No internet (~7.4%) → lowest churn

Checking by senior status

In [10]:
senior_summary = df.groupby('SeniorCitizen')['is_churned'].mean().sort_values(ascending=False)
senior_summary

SeniorCitizen
True     0.416813
False    0.236062
Name: is_churned, dtype: float64

Interpretation

Senior citizens (~41.7%) churn much more than non-seniors (~23.6%)

That’s a ~18 point gap, which is big in churn terms

TASK 4 Revenue-at-Risk

Business Question

How much monthly revenue is at risk due to churn?

In [11]:
churned = df[df['is_churned'] == True]

revenue_at_risk = churned['MonthlyCharges'].sum().round(2)

total_monthly_revenue = df['MonthlyCharges'].sum().round(2)

pct_at_risk = (revenue_at_risk / total_monthly_revenue) * 100

pct_at_risk.round(2)

np.float64(30.5)

Interpretation

30.5% of monthly revenue is at risk at the moment

Task 4 — Segmented Revenue at Risk

In [12]:
monthly_revenue_per_contract = df.groupby('Contract')['MonthlyCharges'].sum()

churned_revenue_per_contract = churned.groupby('Contract')['MonthlyCharges'].sum()

summary = (churned_revenue_per_contract / monthly_revenue_per_contract ) * 100

summary



Contract
Month-to-month    46.968460
One year          14.734868
Two year           4.043751
Name: MonthlyCharges, dtype: float64

The most of the revenue at risk comes logically from month-to-month contracts, as they are the most common

TASK 5 — High-Risk Customer Count and Revenue Mix

Business question: If we focus retention on the highest-risk segment, how many customers and how much monthly revenue are we talking about?

Defining the risk factors

In [13]:
df['high_risk'] = (df['Contract'] == 'Month-to-month') & (df['InternetService'] == 'Fiber optic')

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,is_churned,has_positive_charges,high_risk
0,7590-VHVEG,Female,False,Yes,No,1,No,No phone service,DSL,No,...,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,False,True,False
1,5575-GNVDE,Male,False,No,No,34,Yes,No,DSL,Yes,...,No,One year,No,Mailed check,56.95,1889.50,No,False,True,False
2,3668-QPYBK,Male,False,No,No,2,Yes,No,DSL,Yes,...,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,True,True,False
3,7795-CFOCW,Male,False,No,No,45,No,No phone service,DSL,Yes,...,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,False,True,False
4,9237-HQITU,Female,False,No,No,2,Yes,No,Fiber optic,No,...,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,True,True,True


In [14]:
risk_customers = df[df['high_risk'] == True]
total_high_risk = risk_customers['customerID'].count().round(2)
total_customers = df['customerID'].count().round(2)

high_risk_pct = (total_high_risk / total_customers) * 100

high_risk_pct.round(2)

np.float64(30.21)

30.21 % of clients are high_risk customers

In [15]:
high_risk_who_churned = risk_customers[risk_customers['is_churned'] == True]
total_high_risk_churn = high_risk_who_churned['customerID'].count().round(2)

pct_churn = (total_high_risk_churn / total_high_risk) * 100

pct_churn.round(2)

np.float64(54.61)

54.61 % of high risk customeres have churned

In [16]:
total_monthly_high_risk_revenue = risk_customers['MonthlyCharges'].sum().round(2)

churned_monthly_high_risk_revenue = high_risk_who_churned['MonthlyCharges'].sum().round(2)

pct_of_total_high_risk_revenue = (churned_monthly_high_risk_revenue / total_monthly_high_risk_revenue) * 100

pct_of_total_high_risk_revenue.round(2)

np.float64(54.26)

Interpretation of 54.26%

54.26% is: % of high-risk segment revenue coming from churned customers

What this means in business terms --> More than half of the monthly revenue generated by the high-risk segment is already churned

This segment is not just large — it is financially unstable